In [2]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta
import requests

In [6]:
url = "https://api.cryptowat.ch/exchanges"
response = requests.get(url).json()["result"]
exchanges = [res["symbol"] for res in response]
exchanges
    

['gateio',
 'binance-us',
 'kraken-futures',
 'gemini',
 'liquid',
 'okex',
 'hitbtc',
 'cbot',
 'cexio',
 'btc-china',
 'poloniex',
 'bitbay',
 'bitmex',
 'quoine',
 'bitflyer',
 'okcoin',
 '796',
 'luno',
 'bisq',
 'bittrex',
 'deribit',
 'bitfinex',
 'wex',
 'cryptsy',
 'bitvc',
 'cme',
 'nymex',
 'bitstamp',
 'mtgox',
 'quadriga',
 'dex-aggregated',
 'coinbase-pro',
 'vault-of-satoshi',
 'bithumb',
 'coinone',
 'ftx',
 'uniswap-v2',
 'ftx-us',
 'bitz',
 'comex',
 'kraken',
 'mexbt',
 'huobi',
 'binance']

In [3]:
def fetch_candles(pair: str, before: int, after: int, periods: int, save: bool = False) -> pd.DataFrame:
    '''
    exchange: ftx
    pair: fttusd-perpetual-futures
    '''
    url = f"https://api.cryptowat.ch/markets/ftx/{pair}/ohlc"
    query = {'before': before, 'after': after, 'periods': periods}
    candles = requests.get(url, params=query).json()["result"][f"{periods}"]

    t, o, h, l, c, v = [], [], [], [], [], []
    for candle in candles:
        t.append(datetime.fromtimestamp(candle[0]))
        o.append(candle[1])
        h.append(candle[2])
        l.append(candle[3])
        c.append(candle[4])
        v.append(candle[5])

    candles = pd.DataFrame({"Date": t, "Open": o, "High": h, "Low": l, "Close": c, "Volume": v}).set_index("Date")

    if save:
        pth = f"../data/{periods}/{pair}/"
        os.makedirs(pth, exist_ok=True)
        candles.to_csv(pth + f"{candles.index[0].strftime('%Y-%m-%d')}.csv")

    return candles

In [4]:
pair = "btcusd-perpetual-futures"
current_time = datetime.fromtimestamp(int(time.time()))
after = current_time - timedelta(days=300)
print(current_time)
print(after)
before = int(datetime.timestamp(current_time))
after = int(datetime.timestamp(after))
periods = 60 * 60
print(before, after)
print(before, after + 60 * 60 * 24)

candles = fetch_candles(pair, before, after, periods)
print(candles)

2021-08-31 15:35:46
2020-11-04 15:35:46
1630391746 1604471746
1630391746 1604558146
                        Open     High      Low    Close     Volume
Date                                                              
2020-12-24 17:00:00  22940.0  23210.0  22835.0  23176.0  1725.9189
2020-12-24 18:00:00  23176.0  23490.0  23065.0  23458.5  2925.8211
2020-12-24 19:00:00  23458.5  23481.0  23259.0  23346.0  1013.5225
2020-12-24 20:00:00  23349.0  23350.0  23113.5  23127.0   917.1621
2020-12-24 21:00:00  23126.0  23297.5  23053.0  23219.5  1330.8342
...                      ...      ...      ...      ...        ...
2021-08-31 11:00:00  46865.0  47040.0  46762.0  46904.0  1729.7117
2021-08-31 12:00:00  46903.0  47075.0  46770.0  47034.0  1720.1033
2021-08-31 13:00:00  47034.0  47229.0  46999.0  47156.0  2216.5649
2021-08-31 14:00:00  47156.0  47314.0  47026.0  47066.0  1730.8860
2021-08-31 15:00:00  47066.0  47225.0  47023.0  47158.0  1591.9359

[5999 rows x 5 columns]


In [15]:
pairs = ["btcusd", "ethusd"]
url = "https://api.cryptowat.ch/markets/binance"
markets = requests.get(url).json()["result"]
# from pprint import pprint
# pprint(markets)
for market in markets:
    if "perpetual-futures" in market["pair"]:
        if "dot" in market["pair"]:
            print(market["pair"])
    # if market["pair"] in pairs:
    #     print(market)

dotusdt-perpetual-futures


In [6]:
start = int(datetime.timestamp(datetime(2020, 1, 1)))
# end = int(datetime.timestamp(datetime(2021, 7, 30)))
end = int(datetime.timestamp(datetime.now()))
for pair in pairs:
    fetch_candles(pair, end, start, 60 * 60)